In [4]:
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
!pip install keras-tcn
from tcn import TCN

In [38]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset5/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset5/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset5/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset5/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(66542, 256, 6) (66542, 2) (7600, 256, 6) (7600, 2)


In [39]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [40]:
input_shape = x_train.shape[1:]
inp = Input(shape=x_train.shape[1:])
x = SpatialDropout1D(0.2)(inp)
x = TCN(128,dilations = [1, 2, 4], return_sequences=True)(x)
x = TCN(64,dilations = [1, 2, 4], return_sequences=True)(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
    
conc = concatenate([avg_pool, max_pool])
conc = Dense(64, activation="relu")(conc)
conc = Dropout(0.2)(conc)
outp = Dense(2, activation="softmax")(conc)    
model = Model(inputs=inp, outputs=outp)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 256, 6)]     0           []                               
                                                                                                  
 spatial_dropout1d_8 (SpatialDr  (None, 256, 6)      0           ['input_9[0][0]']                
 opout1D)                                                                                         
                                                                                                  
 tcn_16 (TCN)                   (None, 256, 128)     249728      ['spatial_dropout1d_8[0][0]']    
                                                                                                  
 tcn_17 (TCN)                   (None, 256, 64)      94656       ['tcn_16[0][0]']           

In [41]:
history = model.fit(x_train, y_train, epochs=10, verbose=True, validation_data=(x_validation, y_validation), batch_size=128)

Epoch 1/10
416/416 [==============================] - 110s 258ms/step - loss: 0.6358 - accuracy: 0.7268 - val_loss: 0.4360 - val_accuracy: 0.8049
Epoch 2/10
416/416 [==============================] - 55s 131ms/step - loss: 0.4327 - accuracy: 0.8059 - val_loss: 0.3650 - val_accuracy: 0.8499
Epoch 3/10
416/416 [==============================] - 55s 131ms/step - loss: 0.3934 - accuracy: 0.8295 - val_loss: 0.3483 - val_accuracy: 0.8654
Epoch 4/10
416/416 [==============================] - 54s 131ms/step - loss: 0.3664 - accuracy: 0.8470 - val_loss: 0.2937 - val_accuracy: 0.8841
Epoch 5/10
416/416 [==============================] - 54s 131ms/step - loss: 0.3394 - accuracy: 0.8608 - val_loss: 0.2848 - val_accuracy: 0.8886
Epoch 6/10
416/416 [==============================] - 54s 131ms/step - loss: 0.3272 - accuracy: 0.8664 - val_loss: 0.2819 - val_accuracy: 0.8916
Epoch 7/10
416/416 [==============================] - 54s 131ms/step - loss: 0.3098 - accuracy: 0.8749 - val_loss: 0.2601 - val_a

In [42]:
loss, accuracy = model.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9257
Testing Accuracy: 0.8850
